## Limpieza

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("./presidentes.csv", delimiter=";")

In [ ]:
def clean_ages(age):
    age = age[:2]
    return age

In [ ]:
df["Edad al tomar posesión"] = df["Edad al tomar posesión"].apply(clean_ages)
df["Edad al dejar el cargo[4]\u200b"] = df["Edad al dejar el cargo[4]\u200b"].apply(clean_ages)
df["Edad al morir"] = df["Edad al morir"].apply(clean_ages)

In [ ]:
df.rename(columns={
    "Edad al tomar posesión": "Age in",
    "Edad al dejar el cargo[4]\u200b": "Age out",
    "Edad al morir": "Age dead",
    "Gobernante": "President",
    "Fecha de nacimiento[1]\u200b": "Born Date",
    "Fecha de toma de posesión[2]\u200b": "In Date",
    "Fin del mandato[3]\u200b": "Out Date",
    "Fecha de muerte[5]\u200b": "Dead Date"
}, inplace=True)

In [ ]:
def clean_dates(date):
    if "9999" in date:
        date = "-"
    # Intentar convertir la fecha al formato deseado
    try:
        return pd.to_datetime(date, format='%d/%m/%Y')
    except ValueError:
        # Si la conversión falla, devolver NaT o un valor vacío
        return pd.NaT

In [ ]:
date_columns = ["In Date", "Out Date", "Dead Date", "Born Date"]

df[date_columns] = df[date_columns].map(clean_dates)

In [ ]:
df = df.drop_duplicates(subset='President', keep='first')

# Modificar fecha de muerte de luis echeverria
df.loc[df["President"] == "Luis Echeverría", "Dead Date"] = pd.to_datetime("2022-07-08")

# Edad de muerte de luis echeverria
df.loc[df["President"] == "Luis Echeverría", "Age dead"] = 100

In [ ]:
df.to_excel("presidents.xlsx")
df.to_csv("presidents.csv", index=False)

In [ ]:
df

## Análisis Simple (Ingreso y Fallecimiento)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

file_path = './presidents.csv'
df = pd.read_csv(file_path)

# Limpiar datos no numéricos en columnas de edad
age_in_office_cleaned = pd.to_numeric(df['Age in'], errors='coerce')
age_at_death_cleaned = pd.to_numeric(df['Age dead'], errors='coerce')
presidents = df['President']

# Resetear las escalas de colores
plt.rcParams.update(plt.rcParamsDefault)

# Gráfico 1
plt.figure(figsize=(10, 6))
plt.bar(presidents, age_in_office_cleaned, color='lightgray', label='Edad al asumir')
plt.plot(presidents, age_in_office_cleaned, color='black', label='Edad al asumir (Línea)', linewidth=2, marker='o')
plt.title('Edad de los presidentes al momento de asumir el cargo')
plt.xlabel('Presidentes')
plt.ylabel('Edad')
plt.xticks(rotation=90)
plt.legend()

# Agregar anotaciones
min_age_in = age_in_office_cleaned.min()
min_president_in = df.loc[age_in_office_cleaned.idxmin(), 'President']
max_age_in = age_in_office_cleaned.max()
max_president_in = df.loc[age_in_office_cleaned.idxmax(), 'President']

plt.annotate(f'Mínimo: {min_age_in} ({min_president_in})', 
             xy=(age_in_office_cleaned.idxmin(), min_age_in),
             xytext=(age_in_office_cleaned.idxmin(), min_age_in + 5),
             arrowprops=dict(facecolor='black', arrowstyle="->"))
plt.annotate(f'Máximo: {max_age_in} ({max_president_in})',
             xy=(age_in_office_cleaned.idxmax(), max_age_in),
             xytext=(age_in_office_cleaned.idxmax(), max_age_in - 5),
             arrowprops=dict(facecolor='black', arrowstyle="->"))

plt.tight_layout()
plt.show()

# Gráfico 2
plt.figure(figsize=(10, 6))
plt.bar(presidents, age_at_death_cleaned, color='lightgray', label='Edad al fallecer')
plt.plot(presidents, age_at_death_cleaned, color='black', label='Edad al fallecer (Línea)', linewidth=2, marker='o')
plt.title('Edad de los presidentes al momento de fallecer')
plt.xlabel('Presidentes')
plt.ylabel('Edad')
plt.xticks(rotation=90)
plt.legend()

# Agregar anotaciones
max_age_death = age_at_death_cleaned.max()
max_president_death = df.loc[age_at_death_cleaned.idxmax(), 'President']
min_age_death = age_at_death_cleaned.min()
min_president_death = df.loc[age_at_death_cleaned.idxmin(), 'President']

plt.annotate(f'Máximo: {max_age_death} ({max_president_death})',
             xy=(age_at_death_cleaned.idxmax(), max_age_death),
             xytext=(age_at_death_cleaned.idxmax(), max_age_death - 5),
             arrowprops=dict(facecolor='black', arrowstyle="->"))

plt.annotate(f'Mínimo: {min_age_death} ({min_president_death})',
             xy=(age_at_death_cleaned.idxmin(), min_age_death),
             xytext=(age_at_death_cleaned.idxmin(), min_age_death + 5),
             arrowprops=dict(facecolor='black', arrowstyle="->"))

plt.tight_layout()
plt.show()


## Análisis conjunto

### Expectativa de vida vs edad de comienzo

In [ ]:
file_path = './life-expectancy.csv' 

le = pd.read_csv(file_path)

# Filtrar solo los datos de México
le_mx = le[le['Entity'] == 'Mexico']

In [ ]:
le_mx

In [ ]:
# Renombar columna de esperanza de vida
le_mx.rename(columns={'Period life expectancy at birth - Sex: all - Age: 0': 'Life Expectancy'}, inplace=True)

In [ ]:
pd.DataFrame(le_mx)

In [ ]:
# Agregar columna de life expectancy a df
# si el año de In Date es mayor o igual al añor de la columna Year de le_mx y no mas alla de 6 años, entonces se agrega la esperanza de vida
# In Date esta en el formato de %yyyy-%mm-%dd
# Year esta en el formato de %yyyy

df['In Date'] = pd.to_datetime(df['In Date'], format='%Y-%m-%d', errors='coerce')

def get_life_expectancy(row):
    year = row['In Date'].year
    le_row = le_mx[le_mx['Year'] == year]
    if le_row.empty:
        return None
    return le_row['Life Expectancy'].values[0]

df['Life Expectancy'] = df.apply(get_life_expectancy, axis=1)

df

In [ ]:
# A partir de que In Date se tiene la esperanza de vida
df[df['Life Expectancy'].notnull()]

In [ ]:
# En que año está registrado porfirio diaz
df[df['President'] == 'Porfirio Díaz']

In [ ]:
plt.figure(figsize=(10, 6))

colors = ['black' if pres == 'Adolfo de la Huerta' else 'gray' for pres in df['President']]
plt.bar(df['President'], df['Age in'], label="Edad de inicio", color=colors, alpha=0.6)

plt.plot(df['President'], df['Life Expectancy'], color='black', label="Esperanza de vida", linewidth=2)

plt.xticks(rotation=90)
plt.xlabel("Presidente")
plt.ylabel("Edad de inicio")
plt.title("Edad de inicio de los presidentes de México y la esperanza de vida")
plt.legend()
plt.tight_layout()

plt.show()


In [ ]:
# Convertir las columnas 'In Date' y 'Out Date' a tipo datetime
df['In Date'] = pd.to_datetime(df['In Date'], format='%Y-%m-%d', errors='coerce')
df['Out Date'] = pd.to_datetime(df['Out Date'], format='%Y-%m-%d', errors='coerce')

# Resetear los parámetros de plt
plt.rcParams.update(plt.rcParamsDefault)

# Calcular la duración del mandato en días
df['Duration'] = (df['Out Date'] - df['In Date']).dt.days

# Graficar la duración del mandato y la esperanza de vida
plt.figure(figsize=(10, 6))

plt.bar(df['President'], df['Duration'], label="Duración del mandato", color='gray', alpha=0.6)

# Configurar etiquetas y título
plt.xticks(rotation=90)
plt.xlabel("Presidente")
plt.ylabel("Duración del mandato (días)")
plt.title("Duración del mandato de los presidentes de México")
plt.legend()

# Ajustar el diseño y mostrar la gráfica
plt.tight_layout()
plt.show()